# Segmenting and Clustering Neighborhoods in Toronto

###### Created by Polin Tsenova

### Import Libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [5]:
!pip install lxml

!pip install html5lib

!pip install bs4

     |████████████████████████████████| 5.5MB 793kB/s eta 0:00:01
     |████████████████████████████████| 122kB 5.0MB/s eta 0:00:01     |█████████████████████████▌      | 92kB 4.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [19]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Canada_df=pd.read_html(url, header=0, flavor='html5lib')[0]

In [20]:
Canada_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [21]:
Canada_df.shape

(180, 3)

In [22]:
Canada_df = Canada_df[Canada_df['Borough'] != 'Not assigned']

In [23]:
Canada_df.shape

(103, 3)

In [24]:
# Merge cells having the same postal code
Canada_df_new = Canada_df.groupby('Postal Code')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
Canada_df_new = Canada_df_new.reset_index(drop=False)
Canada_df_new.rename(columns={'Neighbourhood':'Neighbourhood_j'}, inplace=True)

df_merge = pd.merge(Canada_df, Canada_df_new, on='Postal Code', how='outer')
df_merge.drop(['Neighbourhood_j'], axis=1, inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.head()

print(df_merge.shape)
df_merge.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Geocoder

In [27]:
!pip install beautifulsoup4 geocoder folium
import geocoder # import geocoder

geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
#geo_df.rename(columns={'Postal Code':'Postalcode'}, inplace=True)
geo_merge = pd.merge(df_merge, geo_df, on='Postal Code', how='outer')
geo_merge.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Clustering Neighbour Data

In [45]:
toronto_df = geo_merge[geo_merge['Borough'].str.contains("Toronto")]
toronto_df.reset_index(drop=True, inplace=True)

toronto_df.head(15)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [33]:
# Create Toronto map
Toronto = folium.Map(location=[43.65, -79.38], zoom_start=11)

# Add markers
for lat, lng, borough, neighborhood in \
    zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='green',
        fill=True,
        fill_color='orange',
        fill_opacity=0.8,
        parse_html=False).add_to(Toronto)  
    
Toronto

In [34]:
CLIENT_ID = 'PTRITRNUBQPLD1BUWRGYPH1OMCITWZ2PO5L34PYC21MOKEGK' # your Foursquare ID
CLIENT_SECRET = 'ISWCXTWPQSWSDHBYRRFROLMHYSWQIB2ONWDGDZBAFXMBKZHB' # your Foursquare Secret
VERSION = '20210124' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PTRITRNUBQPLD1BUWRGYPH1OMCITWZ2PO5L34PYC21MOKEGK
CLIENT_SECRET:ISWCXTWPQSWSDHBYRRFROLMHYSWQIB2ONWDGDZBAFXMBKZHB


### Explore Neighbourhood

In [44]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


In [48]:
easttoronto_data = toronto_df[toronto_df['Borough'] == 'East Toronto'].reset_index(drop=True)
easttoronto_data.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


In [49]:
easttoronto_data.loc[0, 'Neighbourhood']

'The Beaches'

###### Create a map of East Toronto

In [53]:
map_easttoronto = folium.Map(location=[latitude, longitude], zoom_start=3)

# add markers to map
for lat, lng, label in zip(easttoronto_data['Latitude'], easttoronto_data['Longitude'], easttoronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_easttoronto)  
    
map_easttoronto

NameError: name 'latitude' is not defined

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)